# Protein Structure Data

在定义数据集之前，我们首先需要定义我们想要对蛋白质进行的转换。我们考虑两种转换。 (1) 为了降低内存成本，截断过长的蛋白质序列是一种常见的做法。在TorchProtein中，我们可以通过指定max_length参数来定义蛋白质截断转换，以及通过random参数来确定是从随机残基还是从第一个残基开始截断。 (2) 此外，由于我们希望使用残基特征作为基于结构的模型的节点特征，我们还需要为蛋白质定义一个视图变换。在数据集构建过程中，我们可以将两种转换的组合作为参数传递。

In [1]:
from torchdrug import transforms

truncate_transform = transforms.TruncateProtein(max_length=350, random=False)
protein_view_transform = transforms.ProteinView(view="residue")
transform = transforms.Compose([truncate_transform, protein_view_transform])

为了在本教程中进行高效的计算，我们基于datasets.EnzymeCommission定义了一个小型蛋白质结构数据集EnzymeCommissionToy。

In [2]:
from torchdrug import datasets

class EnzymeCommissionToy(datasets.EnzymeCommission):
    url = "https://miladeepgraphlearningproteindata.s3.us-east-2.amazonaws.com/data/EnzymeCommission.tar.gz"
    md5 = "728e0625d1eb513fa9b7626e4d3bcf4d"
    processed_file = "enzyme_commission_toy.pkl.gz"
    test_cutoffs = [0.3, 0.4, 0.5, 0.7, 0.95]

然后，我们根据这个子类实例化一个蛋白质结构数据集。在第一次实例化时，我们将保存一个名为enzyme_commission_toy.pkl.gz的压缩pickle文件，该文件将所有蛋白质结构数据存储到本地存储中。未来的实例化将直接加载这个pickle文件，因此速度会快得多。酶委员会数据集为每个蛋白质注解了538个二进制功能标签。

In [3]:
import time

start_time = time.time()
dataset = EnzymeCommissionToy("~/protein-datasets/", transform=transform, atom_feature=None,
                            bond_feature=None)
end_time = time.time()
print("Duration of first instantiation: ", end_time - start_time)

start_time = time.time()
dataset = EnzymeCommissionToy("~/protein-datasets/", transform=transform, atom_feature=None,
                            bond_feature=None)
end_time = time.time()
print("Duration of second instantiation: ", end_time - start_time)

train_set, valid_set, test_set = dataset.split()
print("Shape of function labels for a protein: ", dataset[0]["targets"].shape)
print("train samples: %d, valid samples: %d, test samples: %d" % (len(train_set), len(valid_set), len(test_set)))

19:20:47   Extracting /home/weibin/protein-datasets/EnzymeCommission.tar.gz to /home/weibin/protein-datasets


Loading /home/weibin/protein-datasets/EnzymeCommission/enzyme_commission_toy.pkl.gz: 100%|██████████| 1151/1151 [00:07<00:00, 157.10it/s]


Duration of first instantiation:  8.501194953918457
19:20:55   Extracting /home/weibin/protein-datasets/EnzymeCommission.tar.gz to /home/weibin/protein-datasets


Loading /home/weibin/protein-datasets/EnzymeCommission/enzyme_commission_toy.pkl.gz: 100%|██████████| 1151/1151 [00:07<00:00, 146.52it/s]


Duration of second instantiation:  8.833615779876709
Shape of function labels for a protein:  torch.Size([538])
train samples: 959, valid samples: 97, test samples: 95


# Dynamic Graph Construction

TorchProtein使用RDKit来构建蛋白质图。由RDKit构建的蛋白质图只包含四种类型的键边（即，单键、双键、三键或芳香键）。给定数据集的第一个样本，我们选择出第一个和第二个残基的原子，并可视化它们之间的化学键。

In [4]:
from torchdrug import data

protein = dataset[0]["graph"]
is_first_two = (protein.residue_number == 1) | (protein.residue_number == 2)
first_two = protein.residue_mask(is_first_two, compact=True)
first_two.visualize()

然而，仅仅使用键边，无法充分利用蛋白质的丰富结构信息。在接下来的步骤中，我们将尝试动态重构蛋白质图，以更好地表示蛋白质的结构。

## Step 1: Construct Residue-level Graph

我们以数据集的第一个样本为例。对于这个样本，原始的原子级图包含2956个节点，这对于大多数基于结构的模型（如GNNs）来说是无法承受的。因此，在第一步中，我们希望通过仅保留Alpha碳的节点来减小图的大小，从而构建一个残基级的图。我们通过layers.GraphConstruction和layers.geometry.AlphaCarbonNode模块来实现这个目标。layers.geometry.AlphaCarbonNode将丢弃那些没有Alpha碳的无效残基。

In [5]:
from torchdrug import layers
from torchdrug.layers import geometry

graph_construction_model = layers.GraphConstruction(node_layers=[geometry.AlphaCarbonNode()])

_protein = data.Protein.pack([protein])
protein_ = graph_construction_model(_protein)
print("Graph before: ", _protein)
print("Graph after: ", protein_)

Graph before:  PackedProtein(batch_size=1, num_atoms=[2639], num_bonds=[5368], num_residues=[350])
Graph after:  PackedProtein(batch_size=1, num_atoms=[350], num_bonds=[0], num_residues=[350])


注意: 衍生的残基级图不包含边，因为没有两个Alpha碳通过化学键相连。因此，在接下来的步骤中，我们将尝试向这个残基级图中添加不同类型的边。

## Step 2: Add Spatial Edges

在没有边的残基级图上，我们首先考虑在残基之间添加空间边，其距离在一个空间距离阈值内。此外，我们还会删除在蛋白质序列中彼此靠近的残基之间的空间边，因为这些边与折叠结构的关联较小。我们通过layers.geometry.SpatialEdge模块来实现这一目标。

In [6]:
graph_construction_model = layers.GraphConstruction(node_layers=[geometry.AlphaCarbonNode()],
                                                    edge_layers=[geometry.SpatialEdge(radius=10.0, min_distance=5)])

_protein = data.Protein.pack([protein])
protein_ = graph_construction_model(_protein)
print("Graph before: ", _protein)
print("Graph after: ", protein_)

degree = protein_.degree_in + protein_.degree_out
print("Average degree: ", degree.mean().item())
print("Maximum degree: ", degree.max().item())
print("Minimum degree: ", degree.min().item())
print("Number of zero-degree nodes: ", (degree == 0).sum().item())

Graph before:  PackedProtein(batch_size=1, num_atoms=[2639], num_bonds=[5368], num_residues=[350])
Graph after:  PackedProtein(batch_size=1, num_atoms=[350], num_bonds=[4177], num_residues=[350])
Average degree:  23.868572235107422
Maximum degree:  51.0
Minimum degree:  0.0
Number of zero-degree nodes:  5


注意: 仅使用空间边，会有五个节点与图中的任何节点都没有连接，这会阻止在GNN模型中通过这些节点传递信息。在下一步中，我们将尝试通过利用KNN边来解决这个问题。

## Step 3: Add KNN Edges

基于上述的残基级图，我们进一步考虑添加KNN边，其中每个节点将与其K个最近邻节点相连。同时，我们将删除蛋白质序列中彼此靠近的残基之间的KNN边。我们通过使用layers.geometry.KNNEdge模块来实现这一目标。

In [7]:
graph_construction_model = layers.GraphConstruction(node_layers=[geometry.AlphaCarbonNode()],
                                                    edge_layers=[geometry.SpatialEdge(radius=10.0, min_distance=5),
                                                                 geometry.KNNEdge(k=10, min_distance=5)])

_protein = data.Protein.pack([protein])
protein_ = graph_construction_model(_protein)
print("Graph before: ", _protein)
print("Graph after: ", protein_)

degree = protein_.degree_in + protein_.degree_out
print("Average degree: ", degree.mean())
print("Maximum degree: ", degree.max())
print("Minimum degree: ", degree.min())
print("Number of zero-degree nodes: ", (degree == 0).sum())

Graph before:  PackedProtein(batch_size=1, num_atoms=[2639], num_bonds=[5368], num_residues=[350])
Graph after:  PackedProtein(batch_size=1, num_atoms=[350], num_bonds=[5532], num_residues=[350])
Average degree:  tensor(31.6114)
Maximum degree:  tensor(66.)
Minimum degree:  tensor(2.)
Number of zero-degree nodes:  tensor(0)


注意: 在这种情况下，不再存在零度节点。然而，无论是空间边还是KNN边都忽略了蛋白质序列中彼此靠近的残基之间的边。为了补充这种缺失的信息，接下来我们考虑添加顺序边。

## Step 4: Add Sequential Edges

基于上述的残基级图，我们进一步考虑添加顺序边，其中在一个顺序距离阈值内的两个残基将相互连接起来。我们通过使用layers.geometry.SequentialEdge模块来实现这一目标。

In [8]:
graph_construction_model = layers.GraphConstruction(node_layers=[geometry.AlphaCarbonNode()],
                                                    edge_layers=[geometry.SpatialEdge(radius=10.0, min_distance=5),
                                                                 geometry.KNNEdge(k=10, min_distance=5),
                                                                 geometry.SequentialEdge(max_distance=2)])

_protein = data.Protein.pack([protein])
protein_ = graph_construction_model(_protein)
print("Graph before: ", _protein)
print("Graph after: ", protein_)

degree = protein_.degree_in + protein_.degree_out
print("Average degree: ", degree.mean())
print("Maximum degree: ", degree.max())
print("Minimum degree: ", degree.min())
print("Number of zero-degree nodes: ", (degree == 0).sum())

Graph before:  PackedProtein(batch_size=1, num_atoms=[2639], num_bonds=[5368], num_residues=[350])
Graph after:  PackedProtein(batch_size=1, num_atoms=[350], num_bonds=[7276], num_residues=[350])
Average degree:  tensor(41.5771)
Maximum degree:  tensor(76.)
Minimum degree:  tensor(12.)
Number of zero-degree nodes:  tensor(0)


/home/weibin/miniconda3/envs/torchdrug/lib/python3.8/site-packages/torchdrug/layers/functional/functional.py:474: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  index1 = local_index // local_inner_size + offset1


## Overview: Represent Protein Structure as Relational Graph

在进行这样的图构建之后，我们将蛋白质结构表示为一个残基级别的关系图。将空间边和KNN边视为两种类型的边，并将五种不同的顺序距离（即-2，-1，0，1和2）的顺序边视为五种边类型，我们得到了一个具有7种不同边类型的关系图。每条边都与一个40维边特征相关联，该特征是其两个端点的独热残基特征的连接。

In [9]:
nodes_in, nodes_out, edges_type = protein_.edge_list.t()
residue_ids = protein_.residue_type.tolist()
for node_in, node_out, edge_type in zip(nodes_in.tolist()[:5], nodes_out.tolist()[:5], edges_type.tolist()[:5]):
    print("%s -> %s: type %d" % (data.Protein.id2residue[residue_ids[node_in]],
                                 data.Protein.id2residue[residue_ids[node_out]], edge_type))

ILE -> VAL: type 1
TRP -> GLU: type 1
LEU -> GLU: type 1
VAL -> GLU: type 1
ARG -> ASP: type 1


# Protein Structure Representation Model

TorchProtein定义了多种GNN模型，可以作为蛋白质结构编码器。在本教程中，我们将研究优秀的几何感知关系图神经网络（GearNet）及其在我们的小型基准上进行边消息传递扩展的模型（GearNet-Edge）。

## GearNet

GearNet是专门设计用于编码上述定义的残基级别关系图的模型，其关键组成部分是不同残基之间的关系消息传递。在TorchProtein中，我们可以使用models.GearNet来定义一个GearNet模型。

In [10]:
from torchdrug import models

gearnet = models.GearNet(input_dim=21, hidden_dims=[512, 512, 512], num_relation=7,
                         batch_norm=True, concat_hidden=True, short_cut=True, readout="sum")

## GearNet-Edge

GearNet-Edge通过添加边级消息传递来扩展原始的GearNet模型。具体而言，GearNet-Edge构建了线图，其节点是原始图的边，并且它连接了原始图中相邻的边。在此基础上，通过对线图进行关系消息传递来实现边级消息传递。在TorchProtein中，我们可以使用models.GearNet来定义一个GearNet-Edge模型。

In [11]:
gearnet_edge = models.GearNet(input_dim=21, hidden_dims=[512, 512, 512],
                              num_relation=7, edge_input_dim=59, num_angle_bin=8,
                              batch_norm=True, concat_hidden=True, short_cut=True, readout="sum")

# Structure-based Protein Function Prediction

在这个部分，我们将解决小型酶委员会数据集上的蛋白质功能术语预测任务。我们使用GearNet和GearNet-Edge来解决这个任务，并比较它们的性能。

注意: 这个任务旨在预测一个蛋白质是否具有几个特定的功能，其中每个功能可以用一个二进制标签来表示。因此，我们将这个任务形式化为多个二分类任务，并通过多任务学习的方式来共同解决它们。

## Protein Function Prediction with GearNet

我们首先将GearNet模型包装到tasks.MultipleBinaryClassification模块中，该模块可以同时执行所有考虑的二分类任务。在GearNet之上附加了一个MLP预测头，用于生成所有任务的预测结果。

In [12]:
from torchdrug import tasks

graph_construction_model = layers.GraphConstruction(node_layers=[geometry.AlphaCarbonNode()],
                                                    edge_layers=[geometry.SpatialEdge(radius=10.0, min_distance=5),
                                                                 geometry.KNNEdge(k=10, min_distance=5),
                                                                 geometry.SequentialEdge(max_distance=2)],
                                                    edge_feature="gearnet")

task = tasks.MultipleBinaryClassification(gearnet, graph_construction_model=graph_construction_model, num_mlp_layer=3,
                                          task=[_ for _ in range(len(dataset.tasks))], criterion="bce", metric=["auprc@micro", "f1_max"])

现在我们可以训练这个模型了。我们为这个模型设置一个优化器，并将所有内容放在一个Engine实例中。在这个任务上，训练模型进行10个epochs大约需要2分钟的时间。最后，我们对验证集进行评估。

In [13]:
import torch
from torchdrug import core

optimizer = torch.optim.Adam(task.parameters(), lr=1e-4)
solver = core.Engine(task, train_set, valid_set, test_set, optimizer,
                     gpus=[0], batch_size=4)
solver.train(num_epoch=10)
solver.evaluate("valid")

19:21:05   Preprocess training set


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

## Protein Function Prediction with GearNet-Edge

接下来，我们将GearNet-Edge模型包装到tasks.MultipleBinaryClassification模块中，该模块可以同时执行所有考虑的二分类任务。在GearNet-Edge之上附加了一个MLP预测头，用于生成所有任务的预测结果。

In [14]:
graph_construction_model = layers.GraphConstruction(node_layers=[geometry.AlphaCarbonNode()],
                                                    edge_layers=[geometry.SpatialEdge(radius=10.0, min_distance=5),
                                                                 geometry.KNNEdge(k=10, min_distance=5),
                                                                 geometry.SequentialEdge(max_distance=2)],
                                                    edge_feature="gearnet")

task = tasks.MultipleBinaryClassification(gearnet_edge, graph_construction_model=graph_construction_model, num_mlp_layer=3,
                                          task=[_ for _ in range(len(dataset.tasks))], criterion="bce", metric=["auprc@micro", "f1_max"])

我们对模型进行10个epochs的训练，大约需要8分钟的时间，最后在验证集上进行评估。

In [15]:
optimizer = torch.optim.Adam(task.parameters(), lr=1e-4)
solver = core.Engine(task, train_set, valid_set, test_set, optimizer,
                     gpus=[0], batch_size=4)
solver.train(num_epoch=10)
solver.evaluate("valid")

19:23:54   Preprocess training set


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

注意：我们可以观察到，在AUPRC和F1 max方面，GearNet-Edge在这个小型基准测试中表现优于GearNet。然而，这两个模型的性能都不太令人满意，主要是由于数据集的规模过小。我们建议用户在TorchProtein的标准数据集（例如datasets.EnzymeCommission）上应用这两个模型，以更好地研究它们的有效性。